<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%204/4_3_Erster_Blick_in_die_Daten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Installieren aller Pakete

In [ ]:
# Hier die Kaggle Credentials einfügen (ohne Anführungszeichen)

%env KAGGLE_USERNAME=openhpi
%env KAGGLE_KEY=das_ist_der_key

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
warnings.simplefilter('always', category=UserWarning)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import rcParams
import string
rcParams['figure.figsize'] = 14, 10

In [ ]:
def show_example_pictures(X_train, y_train, alphabetic_label):
    p = 0
    for i in range(10):
        plt.subplot(3,5,p+1)
        plt.imshow(X_train[i].reshape(28,28), cmap="gray", interpolation='none')
        plt.title("Label: {}".format(alphabetic_label[y_train[i]]))
        plt.tight_layout()
        p += 1
        
def show_label_frequency(df_train):
    plt.figure(figsize = (20,10)) # Label Count
    sns.set(font_scale=3)
    sns.set_style("whitegrid")
    sns.countplot(df_train['label'], label='medium')
    plt.title("Frequency of each label")

# 4.3 Erster Blick in die Daten

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/cover_sign_language.jpg">



Datensatz: 

### Sign Language MNIST Dataset
Über 34.000 Bilder zu Gebärdensprache ASL (American Sign Language) und dazugehörige Übersetzung der Gesten in das Alphabet in A bis Z. Bilder in 28x28 Pixeln in Graustufen. 

Quelle: kaggle.com

## Download Dataset 

### Manuell
via https://www.kaggle.com/datamunge/sign-language-mnist

### Via API

Hinzufügen der kaggle.json
Speichern als ~/.kaggle/kaggle.json auf Linux, OSX, oder anderen UNIX-basierten Betriebssystemen und unter C:\Users<Windows-username>.kaggle\kaggle.json auf Windows

Siehe https://www.kaggle.com/docs/api oder https://github.com/Kaggle/kaggle-api
        
Beispiel:
~/kaggle/kaggle.json

{"username":"openHPI","key":"das_ist_der_key"}

In [ ]:
!pip3 install kaggle
!kaggle datasets download -d datamunge/sign-language-mnist

In [ ]:
import zipfile
with zipfile.ZipFile("sign-language-mnist.zip", 'r') as zip_ref:
    zip_ref.extractall("")

## Übersicht über alle Dateien

Zunächst sehen wir uns alle Dateien in diesem Datensatz einmal an. In dieser Woche wollen wir aus Bildern Gebärdensprache erkennen und diese in das Alphabet von A bis Z übersetzen. Dafür wechseln wir zunächst in das Verzeichnis mit den Daten und lassen uns alle Dateien auflisten. 

In [ ]:
!ls -gnG
!cd sign_mnist_train

# Grundlagen Gebärdensprachen

Die Amerikanische Gebärdensprache (American Sign Language, kurz ASL) ist eine vollständige, natürliche Sprache, die dieselben linguistischen Eigenschaften wie gesprochene Sprachen aufweist, deren Grammatik sich jedoch vom Englischen unterscheidet. ASL wird durch Bewegungen der Hände und des Gesichts ausgedrückt. Sie ist die Hauptsprache vieler Nordamerikaner, die taub oder schwerhörig sind und wird auch von vielen hörenden Menschen verwendet. 

Die American Sign Language unterscheidet sich durchaus deutlich von der deutschen Gebärdensprache. 


<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/sign.png">

# Datensatz Sign Languag MNIST

Als ersten Schritt werden wir uns zunächst eine der beiden Dateien mit Bilddaten etwas genauer ansehen. Jeweils **sign_mnist_train.csv** und **sign_mnist_test.csv** enthalten die Bilddaten. Hier wurde die Teilung in Train- und Test-Daten, die wir in einer der vorherigen Einheiten bereits angesprochen haben, bereits vorgenommen.

Wenn wir uns allerdings die Dateien einmal als DataFrame einlesen, so fällt auf, dass die einzelnen Pixel jeweils in unterschiedlichen Spalten gespeichert werden und nicht jedes Bild in einer eigenen Datei abgelegt ist. So ist also eine Reihe ein Bild mit insgesamt 284 Pixel - das bildest zusammen ein 28x28 Pixel Bild. Zudem sind die Bilder nur in Graustufen verfügbar. Im Vergleich zur Auflösung in modernen Bildschirmen mit mehreren Tausend Pixeln auf beiden Achsen ist das eine sehr geringe Auflösung. 

In [ ]:
import pandas as pd
df_train = pd.read_csv("sign_mnist_train.csv")
df_test = pd.read_csv("sign_mnist_test.csv")

In [ ]:
df_train.head()

# Ein Beispielbild

Im Folgenden sehen wir uns ein Beispielbild aus dem Datensatz einmal genauer an. Vorher müssen wir jedoch die einzelnen Pixel Werte wieder in die passende Form bringen. 

Die erste Spalte beinhaltet die Labels für die einzelnen Bilder. Die Bilddaten speichern wir jeweils in **X_train** und **X_test**, jedoch ohne das Label. 

In [ ]:
X_train = df_train.drop(["label"], axis=1).values
X_test = df_test.drop(["label"], axis=1).values

Die Label speichern wir in den Variablen **y_train** und **y_test**. 

In [ ]:
y_test = df_test["label"]
y_train = df_train["label"]

Wie wir in der Theorie-Einheit gelernt haben, iterieren Convolutional Neural Networks (CNNs) über 2D Matrizen (3D wenn man die einzelnen Channels betrachtet). Daher müssen wir unsere Bilder auch in dieses Format bringen. Hier auch in 3 Dimensionen, wobei die dritte Dimension nur einen Wert für die Graustufe von 0 bis 255 enthält. 

<img width=40% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/cnn.gif">

In [ ]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [ ]:
X_train.shape

Die Labels in unserem Datensatz sind aktuell nach wie vor noch nur Zahlen. 

In [ ]:
np.sort(df_train["label"].unique())

Da wir auch überprüfen wollen, ob die Bilder in die richtigen Buchstaben aus dem Alphabet übersetzt werden, erstellen wir eine Liste um die Indices jeweils in die Buchstaben zu verwandeln. 

In [ ]:
alphabetic_label = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

Im Folgenden werden wir uns einmal einige Beispielbilder ansehen aus dem Datensatz. Hier sieht man noch einmal, dass die Bilder eine relativ geringe Auflösung haben.  

In [ ]:
show_example_pictures(X_train, y_train, alphabetic_label)

Insgesamt hat auch das ASL-Alphabet 26 verschiedene Buchstaben. Geben wir uns allerdings einmal die Anzahl der Labels aus. Hierbei fällt auf, dass wir nur 24 Labels haben. Das liegt daran, dass der Buchstabe **J** und **Z** mit einer Bewegung dargestellt werden. Daher sind diese nicht Teil des Datensatzes.

In [ ]:
df_train["label"].nunique()

Als letzten Schritt dieser Einheit sehen wir uns einmal die Verteilung der verschiedenen Buchstaben im Datensatz an. Hierfür erstellen wir einen sogenannten Barplot. 

In [ ]:
show_label_frequency(df_train)

Hier sehen wir, dass für jeden Buchstaben zwischen circa 1.200 und 1.000 Bilder im Datensatz vorhanden sind. Die verschiedenen Klassen, in die wir später die Bilder klassifizieren wollen, sind also relativ gleich verteilt. Hier fällt auch noch einmal auf, dass Label 9 und 25 fehlen (die Buchstaben **J** und **Z**). 

Das war es auch schon zur Einheit **Erster Blick in die Daten**. In der nächsten Einheit werden wir versuchen mit einem Convolutional Neural Network die einzelnen Bilder richtig zu klassifizieren, also jeweils einen Buchstaben zuordnen. 